# Trabajo Práctico N° 2
## Objetivo:

Para cada dispositivo presentado por Jampp, determinar el tiempo que transcurrirá hasta que el mismo aparezca nuevamente en una subasta, y el tiempo hasta que el usuario del mismo decida instalar una nueva aplicación.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import auc, accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
import datetime as dt

## Analizo los datos de la primer ventana

In [2]:
auct = pd.read_csv('data/auctions_ventana1.csv', dtype = {"ref_type_id": np.int8, "source_id": np.int8})

In [3]:
auct.head()

,date,device_id,ref_type_id,source_id
0,2019-04-20 23:57:27.912838,1109595589636746168,7,0
1,2019-04-20 23:57:28.381114,5896614299191635403,1,0
2,2019-04-20 23:57:28.515423,4172466725848941608,1,0
3,2019-04-20 23:57:28.700884,2616279795187318849,7,0
4,2019-04-20 23:57:28.868312,8034952072073026056,1,0


In [4]:
auct['date'] = pd.to_datetime(auct['date'])

In [5]:
auct.head()

,date,device_id,ref_type_id,source_id
0,2019-04-20 23:57:27.912838,1109595589636746168,7,0
1,2019-04-20 23:57:28.381114,5896614299191635403,1,0
2,2019-04-20 23:57:28.515423,4172466725848941608,1,0
3,2019-04-20 23:57:28.700884,2616279795187318849,7,0
4,2019-04-20 23:57:28.868312,8034952072073026056,1,0


## Veo cuantas veces aparece cada dispositivo en una subasta

In [39]:
dfApT = auct.groupby('device_id').agg({'date': 'count'}).reset_index()
dfApT.columns = ['device_id', 'count_subs']

In [40]:
dfApT.head()

,device_id,count_subs
0,41863526108385,35
1,135153013040192,8
2,161514654074162,6
3,181891380775191,1
4,186034136943920,7


## Veo la cantidad de veces que aparece cada dispositivo segun el source_id

In [20]:
auct['apariciones'] = 1

In [33]:
dfA = auct.groupby(['device_id', 'source_id']).agg({'apariciones':'sum'}).unstack(1).fillna(0)
dfA.columns = dfA.columns.droplevel(0)

In [34]:
dfA.columns = ['apariciones_source_0', 'count_src_1', 'count_src_2', 'count_src_3', 'count_src_4', 'count_src_5', 'count_src_6', 'count_src_7', 'count_src_8', 'count_src_9']

In [35]:
dfA.head()

,apariciones_source_0,count_src_1,count_src_2,count_src_3,count_src_4,count_src_5,count_src_6,count_src_7,count_src_8,count_src_9
device_id,,,,,,,,,,
41863526108385,0.0,0.0,0.0,28.0,0.0,3.0,0.0,0.0,4.0,0.0
135153013040192,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161514654074162,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
181891380775191,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
186034136943920,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Veo la cantidad de veces que aparece cada dispositivo en una subasta según el ref_type

In [42]:
auct['ref_type_id'].value_counts()

1    13313564
7     2331211
Name: ref_type_id, dtype: int64

In [46]:
dfApRef = auct.groupby(['device_id', 'ref_type_id']).agg({'apariciones':'sum'}).unstack(1).fillna(0)
dfApRef.columns = dfApRef.columns.droplevel(0)
dfApRef.columns = ['count_ref_1', 'count_ref_7']
dfApRef.head()

,count_ref_1,count_ref_7
device_id,,
41863526108385,35.0,0.0
135153013040192,8.0,0.0
161514654074162,6.0,0.0
181891380775191,1.0,0.0
186034136943920,7.0,0.0


In [ ]:
df_y = dfTrain['label']
df_X = dfTrain.drop('label', axis=1).merge(dfUsers, on='person').drop(['person','label'], axis=1)

GBC = GradientBoostingClassifier(random_state=23, n_estimators=50, min_samples_split=50)
scores = cross_val_score(GBC , df_X, df_y, scoring="roc_auc", cv=5)
scores.mean()